# Imagery for predictors

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=RkGcUq99MYSginUBJpyh4eOr_pf-P_0CRQu-78YnxZI&tc=jnoJjHChVRzwc-a62uc2jvwQGGnhUGAXKM5iyn7LVbs&cc=YZIrS2UIjV5vQOcUAoNd1zn0Duyi-VVmNZKUcaohEEw

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AdQt8qjtl6XSGZtGh519EctWWDtd2Je9rkDzQOxovtbD5m0ffSSQRGcK-ts

Successfully saved authorization token.


In [ ]:
STATEFP = "12"
STATENM = "FLORIDA"

In [ ]:
state = ee.FeatureCollection("TIGER/2016/States").filter(ee.Filter.eq('STATEFP', str(STATEFP)))
counties = ee.FeatureCollection("TIGER/2016/Counties").filter(ee.Filter.eq('STATEFP', str(STATEFP)))

In [ ]:
elevation = ee.Image('USGS/NED').select('elevation')
slope = ee.Terrain.slope(elevation).clip(state)

In [ ]:
landcover2006 = ee.Image("USGS/NLCD/NLCD2006").select('landcover').clip(state).rename('landcover_2006') 
landcover2016 = ee.Image("USGS/NLCD/NLCD2016").select('landcover').clip(state).rename('landcover_2016')
  
impervious2006 = ee.Image("USGS/NLCD/NLCD2006").select('impervious').clip(state).rename('impervious_2006')
impervious2016 = ee.Image("USGS/NLCD/NLCD2016").select('impervious').clip(state).rename('impervious_2016')

In [ ]:
opticalBands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
thermalBands = ['B10', 'B11']

def maskL8sr(image):
  
  #stock method to remove clouds
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask1 = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  mask2 = image.mask().reduce('min')
  mask3 = image.select(opticalBands).gt(0).And(
          image.select(opticalBands).lt(10000)).reduce('min')
  mask = mask1.And(mask2).And(mask3)

  #perform adjustments to make the numbers more sensical
  return image.select(opticalBands).divide(10000).addBands(
      image.select(thermalBands).divide(10).clamp(273.15, 373.15).subtract(273.15).divide(100)).updateMask(mask)

In [ ]:
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2020-01-01', '2020-12-31').map(maskL8sr);

In [ ]:
def ndvier(image):
  ndvied = image.normalizedDifference(['B5', 'B4']).rename('NDVI');
  return image.addBands(ndvied);

In [ ]:
def ndwier(image):
  ndwied = image.normalizedDifference(['B3', 'B5']).rename('NDWI');
  return image.addBands(ndwied);

In [ ]:
def ndbier(image):
  ndbied = image.normalizedDifference(['B6', 'B5']).rename('NDBI');
  return image.addBands(ndbied);

In [ ]:
ndvied = l8.map(ndvier).map(ndwier).map(ndbier)
ndviest = ndvied.qualityMosaic('NDVI').select('NDVI').clip(state)
ndwiest = ndvied.qualityMosaic('NDWI').select('NDWI').clip(state)
ndbiest = ndvied.qualityMosaic('NDBI').select('NDBI').clip(state)

In [ ]:
nt2016 = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').filterDate('2016-01-01', '2016-12-31').select('avg_rad').qualityMosaic('avg_rad')
nt2021 = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').filterDate('2021-01-01', '2021-12-31').select('avg_rad').qualityMosaic('avg_rad')

In [ ]:
def createTimeBand(img):
  year = ee.Date(img.get('system:time_start')).get('year').subtract(2012);
  return ee.Image(year).byte().addBands(img)

In [ ]:
lights = ee.ImageCollection('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS').select('stable_lights').map(createTimeBand);

In [ ]:
lights_fit = lights.reduce(ee.Reducer.linearFit())
lights_y = lights_fit.select('scale').rename('stablelights_y')
lights_b = lights_fit.select('offset').rename('stablelights_b')

In [ ]:
day = ee.ImageCollection([ee.Image("https://code.earthengine.google.com/?asset=users/asrenninger/hi_day"),
                          ee.Image("https://code.earthengine.google.com/?asset=users/asrenninger/ak_day"),
                          ee.Image("https://code.earthengine.google.com/?asset=users/asrenninger/conus_day")]).mosaic().rename('daytime_pop')
night = ee.ImageCollection([ee.Image("https://code.earthengine.google.com/?asset=users/asrenninger/hi_night"),
                            ee.Image("https://code.earthengine.google.com/?asset=users/asrenninger/ak_night"),
                            ee.Image("https://code.earthengine.google.com/?asset=users/asrenninger/conus_night")]).mosaic().rename('nighttime_pop')

In [ ]:
composite = landcover2016.addBands([landcover2006, impervious2016, impervious2006, nt2021, nt2016, lights_y, lights_b, day, night, slope, ndviest, ndwiest, ndbiest])
composite = composite.clip(state)
composite = composite.float()

In [ ]:
desc =  "comp_" + STATEFP
print(desc)
task = ee.batch.Export.image.toDrive(
    image = composite,
    folder = STATENM,
    fileNamePrefix = desc,
    description = desc, 
    crs = 'EPSG:3857',
    region = state.geometry(), 
    scale = 30, 
    maxPixels = 1e13,
  )

task.start()

comp_12


# Legacy code

In [ ]:
countylist = counties.toList(counties.size())

In [ ]:
for g in range(counties.size().getInfo()):  
  COUNTYNM = ee.Feature(countylist.get(g)).get("NAME").getInfo()
  desc =  str(STATENM) + '_comp_' + str(COUNTYNM)
  print(desc)
  task = ee.batch.Export.image.toDrive(
    image = composite,
    folder = STATENM,
    fileNamePrefix = desc,
    description = desc, 
    crs = 'EPSG:3857',
    region = ee.Feature(countylist.get(g)).geometry(), 
    scale = 30, 
    maxPixels = 1e13,
  )

  task.start()

ARIZONA_comp_Cochise
ARIZONA_comp_Pinal
ARIZONA_comp_Graham
ARIZONA_comp_Gila
ARIZONA_comp_Navajo
ARIZONA_comp_Coconino
ARIZONA_comp_Yavapai
ARIZONA_comp_Yuma
ARIZONA_comp_Maricopa
ARIZONA_comp_Mohave
ARIZONA_comp_Santa Cruz
ARIZONA_comp_Pima
ARIZONA_comp_Greenlee
ARIZONA_comp_Apache
ARIZONA_comp_La Paz
